# 二十个问题 游戏提示词

In [1]:
import torch
from peft import PeftModel
import json
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from peft import LoraConfig, TaskType, get_peft_model

#在这里修改模型路径
model_path = "/mnt/2b44d1e2-569a-42b3-9610-04ea73771f19/liuqinghua/models/models--Qwen--Qwen2.5-7B-Instruct/snapshots/a09a35458c702b33eeacc393d103063234e8bc28"
model_path = r"D:\model\huggingface\hub\models--Qwen--Qwen2.5-7B-Instruct\snapshots\a09a35458c702b33eeacc393d103063234e8bc28"

In [2]:
#加载角色
characters_list = {}
with open("../characters.json","r",encoding="utf-8") as f:
    characters_list = json.load(f)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path,torch_dtype="auto",device_map = device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
characters_list

{'zhangfei': {'image_path': 'gradio/image/张飞.png',
  'lora_model': 'lora/output/Qwen2.5_instruct_lora_zhangfei/checkpoint-360',
  'zh_name': '张飞'}}

In [28]:
#创建 peft model 的个数要根据已经选择的人物进行选择
peft_model_path =  peft_model_path = r"output/Qwen2.5_instruct_lora_"+ list(characters_list.keys())[0] +"/checkpoint-360"
peft_model = PeftModel.from_pretrained(model,peft_model_path, adapter_name=list(characters_list.keys())[0])
peft_model.eval()
for name in list(characters_list.keys()):
    peft_model_path = r"output/Qwen2.5_instruct_lora_"+name +"/checkpoint-360"
    #peft_model_list.append(PeftModel.from_pretrained(model,peft_model_path, adapter_name=name))
    peft_model.load_adapter(peft_model_path, adapter_name = name)


In [29]:
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (zhangfei): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (zhangfei): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (zhangfei): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_feature

In [88]:
def generate_qwen7B(prompt ,lora=True):
    if lora:
        with torch.no_grad():
            text = tokenizer.apply_chat_template(
            prompt,
            tokenize=False,
            add_generation_prompt=True
            )
            model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
            generated_ids = peft_model.generate(
                **model_inputs,
                max_new_tokens=1024,
            )
            generated_ids = [
                output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
            ]
            response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
            return response
    else:
        with peft_model.disable_adapter():
            text = tokenizer.apply_chat_template(
            prompt,
            tokenize=False,
            add_generation_prompt=True
            )
            model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
            generated_ids = peft_model.generate(
                **model_inputs,
                max_new_tokens=1024,

            )
            generated_ids = [
                output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
            ]
            response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
            return response

In [79]:
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (zhangfei): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (zhangfei): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (zhangfei): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_feature

In [96]:
def llm_generate_keyword( keyword = ""):
    """
    :param  根据游戏规则为词语生成一个非常模糊的描述。
    :return:
    """
    prompt = [
        {
            "role": "system",
            "content": f"""你需要为{keyword}打上一个非常模糊的类别标签（例如带电，纸质，不能吃，白色等宽泛的描述），你只能使用一个词语进行表述，字数少于4个字。描述当中不能与{keyword}有任何重复的字。"""
        }
    ]
    return generate_qwen7B(prompt=prompt,lora=False)

def llm_generate_answer(question = "", keyword = ""):
    """
    :param messages:  回答用户的问题： 是 否 不知道
    :param character_name:
    :return:
    """
    # unload adapter
    prompt = [
    {
        "role": "system",
        "content": f"""关键词：{keyword}。用户的问题是：{question}。
        请你根据关键词来回答用户的问题。你只能回答'是'或者'否'或者'不确定'。
        用户会提出一个与关键词相关的问题。例如关键词是：苹果，用户问题是：“它是红色的吗？” 你需要回答'否'，因为苹果不一定是红色的。用户问题是：“它是否长在树上？” 你需要回答'是'，因为苹果是长在树上的。用户问题是：“它一共有24个品种？” 你需要回答'不知道'，因为这可能超出你的认知了。

        你只能回答'是'或者'否'或者'不知道'。
        """
    }
    ]
    return generate_qwen7B(prompt,lora=False)

def llm_generate_judge(predict = "", keyword = ""):
    """
    :param messages:  回答用户的问题
    :param character_name:
    :return:
    """
    # unload adapter
    prompt = [
    {
        "role": "system",
        "content": f"""你正在主持一个游戏，你负责判断用户的猜测是否符合答案。如果用户的猜测与答案一致（或者是一个东西的不同表达），你需要回复'正确'，否则回复'错误'。如果已经非常靠近答案了，你需要回复'部分正确'。

        例如：
        答案词：优衣库衣服 。用户的猜测是：优衣库的衣服。  回复'正确'。
        答案词：喜马拉雅山 。用户的猜测是：喜马拉雅。  回复'正确'。
        答案词：中国国家图书馆 。用户的猜测是：图书馆。  回复'部分正确'。
        答案词：王老吉 。用户的猜测是：加多宝。  回复'错误'。

        下面请你判断用户的猜测，你只能回复'正确'、'错误'或者'部分正确'：
        答案词：{keyword}。用户的猜测是：{predict}。"""
    }
    ]
    return generate_qwen7B(prompt,lora=False)

def llm_generate_character_question(character_name ="zhangfei",describe = "", history = ""):
    """
    :param messages:  角色根据历史记录进行提问。
    :param character_name:
    :return:
    """
    # unload adapter
    peft_model.set_adapter(character_name)
    zh_name = characters_list[character_name]["zh_name"]
    prompt = [
        {
            "role": "system",
            "content": f"""你需要扮演三国演义中的{zh_name}，假设你穿越到了古代，你和其他古代角色一起参与“二十问”游戏。你的最终目标是猜出属性包括'{describe}'的一个词语。
            请参考其他古代角色的历史问答信息[{str(history) if history != "" else  describe}]，再提一个能用'是'或'否'回答的问题，确保这个问题能大幅缩小猜测的范围。不要重复已有角度，每次提问都要基于最新信息调整方向，目的是缩小答案的范围。"""
        }
    ]
    return generate_qwen7B(prompt)

def llm_generate_character_predict(character_name ="zhangfei",describe = "", history = ""):
    """
    :param messages:  角色根据历史记录进行猜测。
    :param character_name:
    :return:
    """
    # unload adapter
    peft_model.set_adapter(character_name)
    zh_name = characters_list[character_name]["zh_name"]
    prompt = [
        {
            "role": "system",
            "content": f"""你需要扮演三国演义中的{zh_name}，你需要按照{zh_name}的风格发言，假设你穿越到了古代，你和其他古代角色一起参与“猜谜底”游戏。
            谜底（是一个名词）属性包括'{describe}'。
            请参考与这个谜底相关的历史问答信息[{str(history) if history != "" else  describe}]，逐步推理，猜测谜底应该是哪个词。
            请你使用一个名词来回答。"""
        },
        # {
        #     "role": "system",
        #     "content": f"""你需要扮演三国演义中的{zh_name}，你需要按照{zh_name}的风格发言，假设你穿越到了古代，你和其他古代角色一起参与“二十问”游戏。
        #     你的最终目标是猜出属性包括'{describe}'的一个谜底（是一个名词）。
        #     请参考与这个谜底相关的历史问答信息[{str(history) if history != "" else  describe}]，逐步推理，逐渐缩小猜测的范围。
        #     下面请你开逐步思考，详细的展示思考过程，并且最终给出一个总结。"""
        # },
        # {
        #     "role": "user",
        #     "content":"""
        #
        #     """
        # }
    ]
    return generate_qwen7B(prompt)


def test_():
    messages = {}
    messages["secret"] = "冰箱"
    messages["history"] = """第1轮：
    """
    messages["vote"] = """
        """

    response = character_llm_generate_vote(messages=messages, character_name="zhangfei",character_list = ['刘备','关羽','曹操','诸葛亮'])
    print(response)
def test_2():
    response = character_llm_generate_setter( character_name="zhangfei",keyword="天宫空间站")
    print(response)

if __name__=='__main__':

    if True:
        res = llm_generate_character_predict(describe="蓝色",history=["玄德曰：此物可充饥否？判官曰：非也。云长曰：莫非飞禽走兽乎？判官曰：非也。翼德曰：此乃匠人所造之物乎？判官曰：非也。子龙曰：此物可与苍穹相系乎？判官曰：然也。孔明曰：此乃天地自然之象乎？判官曰：然也。云长曰：吾等肉眼可见乎？判官曰：然也。翼德曰：此物似云雾般无形乎？判官曰：非也。子龙曰：此物之巨，可遮天蔽日乎？判官曰：然也。玄德曰：此物有江河湖海乎？判官曰：然也。云长曰：此物上有生灵乎？判官曰：然也。翼德曰：此物昼夜交替乎？判官曰：然也。子龙曰：此物四季更迭乎？判官曰：然也。孔明曰：此物乃吾等立足之地乎？判官曰：然也。云长曰：此物环绕日轮乎？判官曰：然也。翼德曰：此物有月相伴乎？判官曰：然也。子龙曰：此物乃寰宇唯一乎？判官曰：非也。子龙曰：此物乃大地乎？判官曰：非也。"])
        print(res)
    pass

大地
